# Install and Imports

In [0]:
import pandas as pd
import string
import numpy as np 

# Google Mount

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Loading Datasets

In [0]:
## Change the path according to your file location. The csv files are 
# kindle_df = pd.read_csv('/content/drive/Shared drives/CIS550_Project/Kindle_Book_Dataset.csv')

book_depository_authors_df = pd.read_csv('/content/drive/Shared drives/CIS550_Project/book-depository-dataset/authors.csv')
book_depository_categories_df = pd.read_csv('/content/drive/Shared drives/CIS550_Project/book-depository-dataset/categories.csv')
book_depository_formats_df = pd.read_csv('/content/drive/Shared drives/CIS550_Project/book-depository-dataset/formats.csv')
book_depository_places_df = pd.read_csv('/content/drive/Shared drives/CIS550_Project/book-depository-dataset/places.csv')
book_depository_dataset_df = pd.read_csv('/content/drive/Shared drives/CIS550_Project/book-depository-dataset/dataset.csv')  

nyTimes_df = pd.read_json('/content/drive/Shared drives/CIS550_Project/nyt2.json', lines=True)

goodReads_books_df = pd.read_csv('/content/drive/Shared drives/CIS550_Project/goodReads_books.csv', error_bad_lines=False)

b'Skipping line 3350: expected 12 fields, saw 13\nSkipping line 4704: expected 12 fields, saw 13\nSkipping line 5879: expected 12 fields, saw 13\nSkipping line 8981: expected 12 fields, saw 13\n'


In [0]:
book_depository_dataset_df.columns

Index(['authors', 'bestsellers-rank', 'categories', 'description',
       'dimension-x', 'dimension-y', 'dimension-z', 'edition',
       'edition-statement', 'for-ages', 'format', 'id', 'illustrations-note',
       'imprint', 'index-date', 'isbn10', 'isbn13', 'lang', 'publication-date',
       'publication-place', 'rating-avg', 'rating-count', 'title', 'url',
       'weight'],
      dtype='object')

In [0]:
book_depository_dataset_df.isna().sum()

authors                     0
bestsellers-rank       514477
categories                  0
description            103248
dimension-x             56035
dimension-y             97327
dimension-z             56035
edition                908080
edition-statement      725007
for-ages              1027356
format                   3898
id                          0
illustrations-note     712334
imprint                814479
index-date                  0
isbn10                      0
isbn13                  17447
lang                    23687
publication-date         4721
publication-place      327659
rating-avg             458907
rating-count           458907
title                       0
url                         0
weight                  95833
dtype: int64

# Selecting columns from Book Depository Dataset

In [0]:
books_df = book_depository_dataset_df[['isbn10', 'title', 'description', 'format','url', 'imprint','publication-date', 'publication-place', 'rating-avg', 'lang']]
books_df.rename(columns={'imprint': 'publisher'}, inplace = True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [0]:
books_df = books_df.dropna()
# books_df.isna().sum()
# print(len(books_df.dropna()))

# b = books_df['isbn10'].apply(lambda x: len(x))
# books_df[b != 10]
display(len(pd.merge(books_df, book_depository_dataset_df[['isbn10', 'for-ages']], left_on='isbn10', right_on='isbn10', how='inner')))

181653

In [0]:
books_df.columns

Index(['isbn10', 'title', 'description', 'format', 'url', 'publisher',
       'publication-date', 'publication-place', 'rating-avg', 'lang'],
      dtype='object')

In [0]:
books_df

,isbn10,title,description,format,url,publisher,publication-date,publication-place,rating-avg,lang
4,321934075,"The Truthful Art : Data, Charts, and Maps for ...","No matter what your actual job title, you are-...",1,/Truthful-Art-Alberto-Cairo/9780321934079,New Riders Publishing,2016-02-28,4.0,4.30,en
5,812995708,Dreyer's English : An Utterly Correct Guide to...,"NEW YORK TIMES BESTSELLER - A sharp, funny gra...",3,/Dreyers-English-Benjamin-Dreyer/9780812995701,Ballantine Books Inc.,2019-02-05,1.0,4.36,en
10,134545133,The Adobe Photoshop Lightroom Classic CC Book ...,"Since Lightroom first launched 11 years ago, S...",1,/Adobe-Photoshop-Lightroom-Classic-CC-Book-for...,New Riders Publishing,2018-07-01,4.0,4.45,en
16,1430224894,Build Your Own CNC Machine,Do you like to build things? Are you ever frus...,1,/Build-Your-Own-CNC-Machine-James-Floyd-Kelly/...,aPress,2009-11-30,8.0,3.87,en
17,8134367,Spelling Ages 5-6: New Edition,Level: KS1,1,/Spelling-Ages-5-6-New-Edition-Collins-Easy-Le...,Collins,2015-09-01,2.0,3.83,en
...,...,...,...,...,...,...,...,...,...,...
1086937,1590176219,Proper Doctoring,"""People come to us for help. They come for hea...",1,/Proper-Doctoring-David-Mendel/9781590176214,NYRB Classics,2013-09-24,9.0,3.88,en
1086938,415195624,Using Groups to Help People,This new edition of Using Groups to Help Peopl...,1,/Using-Groups-Help-People-Dorothy-Stock-Whitak...,ROUTLEDGE,2001-01-31,1.0,4.50,en
1086940,367437481,Dangerous Decade : Taiwan's Security and Crisi...,Taiwan's position looks increasingly precariou...,1,/Dangerous-Decade-Brendan-Taylor/9780367437480,ROUTLEDGE,2019-09-18,1.0,4.66,en
1086942,1781257736,Shapeshifters : A Doctor's Notes on Medicine &...,Unreliable bodies and shifting symptoms are al...,2,/Shapeshifters-Gavin-Francis/9781781257739,Wellcome Collection,2018-05-03,1.0,3.93,en


# Getting the publication places

In [0]:
books_df = pd.merge(books_df, book_depository_places_df, left_on='publication-place', right_on='place_id', how='left')

In [0]:
books_df

,isbn10,title,description,format,url,publisher,publication-date,publication-place,rating-avg,lang,place_id,place_name
0,321934075,"The Truthful Art : Data, Charts, and Maps for ...","No matter what your actual job title, you are-...",1,/Truthful-Art-Alberto-Cairo/9780321934079,New Riders Publishing,2016-02-28,4.0,4.30,en,4.0,"Rolindale, MA, United States"
1,812995708,Dreyer's English : An Utterly Correct Guide to...,"NEW YORK TIMES BESTSELLER - A sharp, funny gra...",3,/Dreyers-English-Benjamin-Dreyer/9780812995701,Ballantine Books Inc.,2019-02-05,1.0,4.36,en,1.0,"London, United Kingdom"
2,134545133,The Adobe Photoshop Lightroom Classic CC Book ...,"Since Lightroom first launched 11 years ago, S...",1,/Adobe-Photoshop-Lightroom-Classic-CC-Book-for...,New Riders Publishing,2018-07-01,4.0,4.45,en,4.0,"Rolindale, MA, United States"
3,1430224894,Build Your Own CNC Machine,Do you like to build things? Are you ever frus...,1,/Build-Your-Own-CNC-Machine-James-Floyd-Kelly/...,aPress,2009-11-30,8.0,3.87,en,8.0,"Rockport, ME, United States"
4,8134367,Spelling Ages 5-6: New Edition,Level: KS1,1,/Spelling-Ages-5-6-New-Edition-Collins-Easy-Le...,Collins,2015-09-01,2.0,3.83,en,2.0,"North Adams, United States"
...,...,...,...,...,...,...,...,...,...,...,...,...
181648,1590176219,Proper Doctoring,"""People come to us for help. They come for hea...",1,/Proper-Doctoring-David-Mendel/9781590176214,NYRB Classics,2013-09-24,9.0,3.88,en,9.0,"New York, United States"
181649,415195624,Using Groups to Help People,This new edition of Using Groups to Help Peopl...,1,/Using-Groups-Help-People-Dorothy-Stock-Whitak...,ROUTLEDGE,2001-01-31,1.0,4.50,en,1.0,"London, United Kingdom"
181650,367437481,Dangerous Decade : Taiwan's Security and Crisi...,Taiwan's position looks increasingly precariou...,1,/Dangerous-Decade-Brendan-Taylor/9780367437480,ROUTLEDGE,2019-09-18,1.0,4.66,en,1.0,"London, United Kingdom"
181651,1781257736,Shapeshifters : A Doctor's Notes on Medicine &...,Unreliable bodies and shifting symptoms are al...,2,/Shapeshifters-Gavin-Francis/9781781257739,Wellcome Collection,2018-05-03,1.0,3.93,en,1.0,"London, United Kingdom"


In [0]:
books_df.drop(['publication-place','place_id'],axis=1, inplace=True)

# Getting the format of the book

In [0]:
book_depository_formats_df_copy = book_depository_formats_df.copy()
book_depository_formats_df_copy['format_id'] = book_depository_formats_df_copy['format_id'].astype(str)

In [0]:
books_df = pd.merge(books_df, book_depository_formats_df_copy, left_on='format', right_on='format_id', how='left')

In [0]:
books_df

,isbn10,title,description,format,url,publisher,publication-date,rating-avg,lang,place_name,format_id,format_name
0,321934075,"The Truthful Art : Data, Charts, and Maps for ...","No matter what your actual job title, you are-...",1,/Truthful-Art-Alberto-Cairo/9780321934079,New Riders Publishing,2016-02-28,4.30,en,"Rolindale, MA, United States",1,Paperback
1,812995708,Dreyer's English : An Utterly Correct Guide to...,"NEW YORK TIMES BESTSELLER - A sharp, funny gra...",3,/Dreyers-English-Benjamin-Dreyer/9780812995701,Ballantine Books Inc.,2019-02-05,4.36,en,"London, United Kingdom",3,Spiral
2,134545133,The Adobe Photoshop Lightroom Classic CC Book ...,"Since Lightroom first launched 11 years ago, S...",1,/Adobe-Photoshop-Lightroom-Classic-CC-Book-for...,New Riders Publishing,2018-07-01,4.45,en,"Rolindale, MA, United States",1,Paperback
3,1430224894,Build Your Own CNC Machine,Do you like to build things? Are you ever frus...,1,/Build-Your-Own-CNC-Machine-James-Floyd-Kelly/...,aPress,2009-11-30,3.87,en,"Rockport, ME, United States",1,Paperback
4,8134367,Spelling Ages 5-6: New Edition,Level: KS1,1,/Spelling-Ages-5-6-New-Edition-Collins-Easy-Le...,Collins,2015-09-01,3.83,en,"North Adams, United States",1,Paperback
...,...,...,...,...,...,...,...,...,...,...,...,...
181648,1590176219,Proper Doctoring,"""People come to us for help. They come for hea...",1,/Proper-Doctoring-David-Mendel/9781590176214,NYRB Classics,2013-09-24,3.88,en,"New York, United States",1,Paperback
181649,415195624,Using Groups to Help People,This new edition of Using Groups to Help Peopl...,1,/Using-Groups-Help-People-Dorothy-Stock-Whitak...,ROUTLEDGE,2001-01-31,4.50,en,"London, United Kingdom",1,Paperback
181650,367437481,Dangerous Decade : Taiwan's Security and Crisi...,Taiwan's position looks increasingly precariou...,1,/Dangerous-Decade-Brendan-Taylor/9780367437480,ROUTLEDGE,2019-09-18,4.66,en,"London, United Kingdom",1,Paperback
181651,1781257736,Shapeshifters : A Doctor's Notes on Medicine &...,Unreliable bodies and shifting symptoms are al...,2,/Shapeshifters-Gavin-Francis/9781781257739,Wellcome Collection,2018-05-03,3.93,en,"London, United Kingdom",2,Hardback


In [0]:
books_df.drop(['format_id','format'],axis=1, inplace=True)

# Merging with Book Cover to get image urls

In [0]:
book_cover_df = pd.read_csv('/content/drive/Shared drives/CIS550_Project/finalCSV/bookCover.csv')

In [0]:
display(pd.merge(books_df, book_cover_df[['ISBN', 'Image-URL-L']], left_on='isbn10', right_on='ISBN', how='left'))

,isbn10,title,description,url,publisher,publication-date,rating-avg,lang,place_name,format_name,ISBN,Image-URL-L
0,321934075,"The Truthful Art : Data, Charts, and Maps for ...","No matter what your actual job title, you are-...",/Truthful-Art-Alberto-Cairo/9780321934079,New Riders Publishing,2016-02-28,4.30,en,"Rolindale, MA, United States",Paperback,NaN,NaN
1,812995708,Dreyer's English : An Utterly Correct Guide to...,"NEW YORK TIMES BESTSELLER - A sharp, funny gra...",/Dreyers-English-Benjamin-Dreyer/9780812995701,Ballantine Books Inc.,2019-02-05,4.36,en,"London, United Kingdom",Spiral,NaN,NaN
2,134545133,The Adobe Photoshop Lightroom Classic CC Book ...,"Since Lightroom first launched 11 years ago, S...",/Adobe-Photoshop-Lightroom-Classic-CC-Book-for...,New Riders Publishing,2018-07-01,4.45,en,"Rolindale, MA, United States",Paperback,NaN,NaN
3,1430224894,Build Your Own CNC Machine,Do you like to build things? Are you ever frus...,/Build-Your-Own-CNC-Machine-James-Floyd-Kelly/...,aPress,2009-11-30,3.87,en,"Rockport, ME, United States",Paperback,NaN,NaN
4,8134367,Spelling Ages 5-6: New Edition,Level: KS1,/Spelling-Ages-5-6-New-Edition-Collins-Easy-Le...,Collins,2015-09-01,3.83,en,"North Adams, United States",Paperback,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
181648,1590176219,Proper Doctoring,"""People come to us for help. They come for hea...",/Proper-Doctoring-David-Mendel/9781590176214,NYRB Classics,2013-09-24,3.88,en,"New York, United States",Paperback,NaN,NaN
181649,415195624,Using Groups to Help People,This new edition of Using Groups to Help Peopl...,/Using-Groups-Help-People-Dorothy-Stock-Whitak...,ROUTLEDGE,2001-01-31,4.50,en,"London, United Kingdom",Paperback,NaN,NaN
181650,367437481,Dangerous Decade : Taiwan's Security and Crisi...,Taiwan's position looks increasingly precariou...,/Dangerous-Decade-Brendan-Taylor/9780367437480,ROUTLEDGE,2019-09-18,4.66,en,"London, United Kingdom",Paperback,NaN,NaN
181651,1781257736,Shapeshifters : A Doctor's Notes on Medicine &...,Unreliable bodies and shifting symptoms are al...,/Shapeshifters-Gavin-Francis/9781781257739,Wellcome Collection,2018-05-03,3.93,en,"London, United Kingdom",Hardback,NaN,NaN


In [0]:
books_df = pd.merge(books_df, book_cover_df[['ISBN', 'Image-URL-L']], left_on='isbn10', right_on='ISBN', how='left')

In [0]:
books_df.rename(columns={'Image-URL-L': 'img_url'}, inplace = True)

In [0]:
books_df.drop(['ISBN'],axis=1, inplace=True)

In [0]:
books_df

,isbn10,title,description,url,publisher,publication-date,rating-avg,lang,place_name,format_name,num_pages,img_url
0,321934075,"The Truthful Art : Data, Charts, and Maps for ...","No matter what your actual job title, you are-...",/Truthful-Art-Alberto-Cairo/9780321934079,New Riders Publishing,2016-02-28,4.30,en,"Rolindale, MA, United States",Paperback,NaN,NaN
1,812995708,Dreyer's English : An Utterly Correct Guide to...,"NEW YORK TIMES BESTSELLER - A sharp, funny gra...",/Dreyers-English-Benjamin-Dreyer/9780812995701,Ballantine Books Inc.,2019-02-05,4.36,en,"London, United Kingdom",Spiral,NaN,NaN
2,134545133,The Adobe Photoshop Lightroom Classic CC Book ...,"Since Lightroom first launched 11 years ago, S...",/Adobe-Photoshop-Lightroom-Classic-CC-Book-for...,New Riders Publishing,2018-07-01,4.45,en,"Rolindale, MA, United States",Paperback,NaN,NaN
3,1430224894,Build Your Own CNC Machine,Do you like to build things? Are you ever frus...,/Build-Your-Own-CNC-Machine-James-Floyd-Kelly/...,aPress,2009-11-30,3.87,en,"Rockport, ME, United States",Paperback,NaN,NaN
4,8134367,Spelling Ages 5-6: New Edition,Level: KS1,/Spelling-Ages-5-6-New-Edition-Collins-Easy-Le...,Collins,2015-09-01,3.83,en,"North Adams, United States",Paperback,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
181648,1590176219,Proper Doctoring,"""People come to us for help. They come for hea...",/Proper-Doctoring-David-Mendel/9781590176214,NYRB Classics,2013-09-24,3.88,en,"New York, United States",Paperback,NaN,NaN
181649,415195624,Using Groups to Help People,This new edition of Using Groups to Help Peopl...,/Using-Groups-Help-People-Dorothy-Stock-Whitak...,ROUTLEDGE,2001-01-31,4.50,en,"London, United Kingdom",Paperback,NaN,NaN
181650,367437481,Dangerous Decade : Taiwan's Security and Crisi...,Taiwan's position looks increasingly precariou...,/Dangerous-Decade-Brendan-Taylor/9780367437480,ROUTLEDGE,2019-09-18,4.66,en,"London, United Kingdom",Paperback,NaN,NaN
181651,1781257736,Shapeshifters : A Doctor's Notes on Medicine &...,Unreliable bodies and shifting symptoms are al...,/Shapeshifters-Gavin-Francis/9781781257739,Wellcome Collection,2018-05-03,3.93,en,"London, United Kingdom",Hardback,NaN,NaN


# Merging with GoodReads to get number of pages

In [0]:
goodReads_books_df

In [0]:
books_df = pd.merge(books_df, goodReads_books_df[['isbn', '  num_pages']], left_on='isbn10', right_on='isbn', how='left')

In [0]:
books_df.rename(columns={'  num_pages': 'num_pages'}, inplace = True)

In [0]:
books_df.drop(['isbn'],axis=1, inplace=True)

In [0]:
books_df

,isbn10,title,description,url,publisher,publication-date,rating-avg,lang,place_name,format_name,num_pages
0,321934075,"The Truthful Art : Data, Charts, and Maps for ...","No matter what your actual job title, you are-...",/Truthful-Art-Alberto-Cairo/9780321934079,New Riders Publishing,2016-02-28,4.30,en,"Rolindale, MA, United States",Paperback,NaN
1,812995708,Dreyer's English : An Utterly Correct Guide to...,"NEW YORK TIMES BESTSELLER - A sharp, funny gra...",/Dreyers-English-Benjamin-Dreyer/9780812995701,Ballantine Books Inc.,2019-02-05,4.36,en,"London, United Kingdom",Spiral,NaN
2,134545133,The Adobe Photoshop Lightroom Classic CC Book ...,"Since Lightroom first launched 11 years ago, S...",/Adobe-Photoshop-Lightroom-Classic-CC-Book-for...,New Riders Publishing,2018-07-01,4.45,en,"Rolindale, MA, United States",Paperback,NaN
3,1430224894,Build Your Own CNC Machine,Do you like to build things? Are you ever frus...,/Build-Your-Own-CNC-Machine-James-Floyd-Kelly/...,aPress,2009-11-30,3.87,en,"Rockport, ME, United States",Paperback,NaN
4,8134367,Spelling Ages 5-6: New Edition,Level: KS1,/Spelling-Ages-5-6-New-Edition-Collins-Easy-Le...,Collins,2015-09-01,3.83,en,"North Adams, United States",Paperback,NaN
...,...,...,...,...,...,...,...,...,...,...,...
181648,1590176219,Proper Doctoring,"""People come to us for help. They come for hea...",/Proper-Doctoring-David-Mendel/9781590176214,NYRB Classics,2013-09-24,3.88,en,"New York, United States",Paperback,NaN
181649,415195624,Using Groups to Help People,This new edition of Using Groups to Help Peopl...,/Using-Groups-Help-People-Dorothy-Stock-Whitak...,ROUTLEDGE,2001-01-31,4.50,en,"London, United Kingdom",Paperback,NaN
181650,367437481,Dangerous Decade : Taiwan's Security and Crisi...,Taiwan's position looks increasingly precariou...,/Dangerous-Decade-Brendan-Taylor/9780367437480,ROUTLEDGE,2019-09-18,4.66,en,"London, United Kingdom",Paperback,NaN
181651,1781257736,Shapeshifters : A Doctor's Notes on Medicine &...,Unreliable bodies and shifting symptoms are al...,/Shapeshifters-Gavin-Francis/9781781257739,Wellcome Collection,2018-05-03,3.93,en,"London, United Kingdom",Hardback,NaN


In [0]:
books_df.drop(['isbn'],axis=1, inplace=True)

In [0]:
books_df.rename(columns={'Image-URL-L': 'img_url'}, inplace = True)

# Saving the Books Dataset

In [0]:
books_df.to_csv('/content/drive/Shared drives/CIS550_Project/Books-temp/updated/books.csv')

# Processing NYTimes Dataset

In [0]:
blah = pd.read_csv('/content/drive/Shared drives/CIS550_Project/finalCSV/NYTimesBestSellers.csv')

In [0]:
blah

,Unnamed: 0,rank,rank_last_week,weeks_on_list,isbn,publisher,description,title,author,book_image,book_image_width,book_image_height,amazon_product_url
0,0,1,3,6,1451627289,Scribner,An English teacher travels back to 1958 by way...,11/22/63,Stephen King,https://s1.nyt.com/du/books/images/97814516272...,328.0,495.0,http://www.amazon.com/11-22-63-A-Novel/dp/1451...
1,1,2,0,1,039915731X,Penguin Group,Jack Ryan Jr. must stop an emerging threat fro...,LOCKED ON,Tom Clancy with Mark Greaney,https://s1.nyt.com/du/books/images/97803991573...,128.0,192.0,http://www.amazon.com/Locked-Jack-Ryan-Tom-Cla...
2,2,3,2,44,0425232204,Penguin Group,A young white woman and two black maids in 196...,THE HELP,Kathryn Stockett,https://s1.nyt.com/du/books/images/97814406976...,330.0,440.0,http://www.amazon.com/The-Help-Kathryn-Stocket...
3,3,4,4,8,0385535139,Knopf Doubleday Publishing,Partners in a small law firm take on a big cas...,THE LITIGATORS,John Grisham,https://s1.nyt.com/du/books/images/97803855351...,184.0,280.0,http://www.amazon.com/The-Litigators-John-Gris...
4,4,5,9,18,0307272117,Knopf Doubleday Publishing,A hacker and a journalist investigate the disa...,THE GIRL WITH THE DRAGON TATTOO,Stieg Larsson,https://s1.nyt.com/du/books/images/97803079494...,277.0,495.0,http://www.amazon.com/Girl-Dragon-Tattoo-Stieg...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3565,3565,6,0,0,1426215649,National Geographic Society,A photographic tour of 225 global destinations.,DESTINATIONS OF A LIFETIME,the staff of National Geographic,https://s1.nyt.com/du/books/images/97814262156...,330.0,417.0,http://www.amazon.com/Destinations-Lifetime-Wo...
3566,3566,7,0,0,0544703391,Houghton Mifflin Harcourt,The creator of the New York Times dialect quiz...,SPEAKING AMERICAN,Josh Katz,https://s1.nyt.com/du/books/images/97805447033...,330.0,314.0,https://www.amazon.com/Speaking-American-Youse...
3567,3567,8,0,0,0307476073,Knopf,"An account of the life-changing 1,100-mile sol...",WILD,Cheryl Strayed,https://s1.nyt.com/du/books/images/97803074760...,321.0,495.0,http://www.amazon.com/Wild-Found-Pacific-Crest...
3568,3568,9,0,0,0399584692,NAL,The author was lost on a train in India at age...,LION,Saroo Brierley,https://s1.nyt.com/du/books/images/97803995846...,330.0,494.0,https://www.amazon.com/Lion-Movie-Tie-Saroo-Br...


In [0]:
blah = blah.dropna()

In [0]:
nytimes_books = blah[['isbn', 'title', 'description', 'publisher', 'amazon_product_url', 'book_image']]

In [0]:
nytimes_books['publication_date'] = None
nytimes_books['num_pages'] = None
nytimes_books['lang'] = None

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [0]:
# nytimes_books.isna().sum()
nytimes_books

,isbn,title,description,publisher,amazon_product_url,book_image,publication_date,num_pages,lang
0,1451627289,11/22/63,An English teacher travels back to 1958 by way...,Scribner,http://www.amazon.com/11-22-63-A-Novel/dp/1451...,https://s1.nyt.com/du/books/images/97814516272...,None,None,None
1,039915731X,LOCKED ON,Jack Ryan Jr. must stop an emerging threat fro...,Penguin Group,http://www.amazon.com/Locked-Jack-Ryan-Tom-Cla...,https://s1.nyt.com/du/books/images/97803991573...,None,None,None
2,0425232204,THE HELP,A young white woman and two black maids in 196...,Penguin Group,http://www.amazon.com/The-Help-Kathryn-Stocket...,https://s1.nyt.com/du/books/images/97814406976...,None,None,None
3,0385535139,THE LITIGATORS,Partners in a small law firm take on a big cas...,Knopf Doubleday Publishing,http://www.amazon.com/The-Litigators-John-Gris...,https://s1.nyt.com/du/books/images/97803855351...,None,None,None
4,0307272117,THE GIRL WITH THE DRAGON TATTOO,A hacker and a journalist investigate the disa...,Knopf Doubleday Publishing,http://www.amazon.com/Girl-Dragon-Tattoo-Stieg...,https://s1.nyt.com/du/books/images/97803079494...,None,None,None
...,...,...,...,...,...,...,...,...,...
3565,1426215649,DESTINATIONS OF A LIFETIME,A photographic tour of 225 global destinations.,National Geographic Society,http://www.amazon.com/Destinations-Lifetime-Wo...,https://s1.nyt.com/du/books/images/97814262156...,None,None,None
3566,0544703391,SPEAKING AMERICAN,The creator of the New York Times dialect quiz...,Houghton Mifflin Harcourt,https://www.amazon.com/Speaking-American-Youse...,https://s1.nyt.com/du/books/images/97805447033...,None,None,None
3567,0307476073,WILD,"An account of the life-changing 1,100-mile sol...",Knopf,http://www.amazon.com/Wild-Found-Pacific-Crest...,https://s1.nyt.com/du/books/images/97803074760...,None,None,None
3568,0399584692,LION,The author was lost on a train in India at age...,NAL,https://www.amazon.com/Lion-Movie-Tie-Saroo-Br...,https://s1.nyt.com/du/books/images/97803995846...,None,None,None


# Using Google Book API to populate NYTimes with number of pages, publication date and language

In [0]:
import requests as req
#  GET https://www.googleapis.com/books/v1/volumes?q=flowers+inauthor:keyes&key=yourAPIKey
# https://www.googleapis.com/books/v1/volumes?q=isbn:0771595158

resp = req.get("https://www.googleapis.com/books/v1/volumes?q=isbn:321934075")
resp = resp.json()

In [0]:
resp

{'kind': 'books#volumes', 'totalItems': 0}

In [0]:
import math
step = 0

item = []
auth = {}
cat = {}

for i, row in nytimes_books.iterrows():

  try:
    resp = req.get("https://www.googleapis.com/books/v1/volumes?q=isbn:"+ row.isbn)
    resp = resp.json()
    
    row.publication_date = resp['items'][0]['volumeInfo'].get('publishedDate')
    row.num_pages = resp['items'][0]['volumeInfo'].get('pageCount')
    row.lang = resp['items'][0]['volumeInfo'].get('language')
    
    authors = resp['items'][0]['volumeInfo'].get('authors')
    auth[row.isbn] = authors

    categories = resp['items'][0]['volumeInfo'].get('categories')
    cat[row.isbn] = categories
  except Exception as e:      
    print(str(e))
    item.append(row)
    continue
  # step += 1
  # if step > 2:
  #   break
  print(i)
  item.append(row)

In [0]:
# df = pd.DataFrame(auth.items(), columns=['isbn', 'author'])
cat_df = pd.DataFrame(cat.items(), columns=['isbn', 'category'])

In [0]:
cat_df.columns

Index(['isbn', 'category'], dtype='object')

In [0]:
# display(df.explode('author'))
# df = df.explode('author')
display(df)

In [0]:
# display(cat_df.explode('category'))
cat_df = cat_df.explode('category')

cat_df

,isbn,category
0,1451627289,Fiction
1,039915731X,Fiction
2,0425232204,Fiction
3,0385535139,Fiction
4,0307272117,Fiction
...,...,...
1547,0374298637,Biography & Autobiography
1548,1426215614,Photography
1549,1426215649,Travel
1550,0399584692,Biography & Autobiography


In [0]:
nytimes_finaldf = pd.DataFrame(item)

In [0]:
nytimes_finaldf.to_csv('/content/drive/Shared drives/CIS550_Project/Books-temp/updated/nytimes.csv')

In [0]:
df.to_csv('/content/drive/Shared drives/CIS550_Project/Books-temp/updated/nytimesbookauthor.csv')

In [0]:
cat_df.to_csv('/content/drive/Shared drives/CIS550_Project/Books-temp/updated/nytimesbookcategory.csv')

# Using Google Book API to populate Book Depository with number of pages and image url

In [0]:
books_df

,isbn10,title,description,url,publisher,publication-date,rating-avg,lang,place_name,format_name,num_pages,img_url
0,321934075,"The Truthful Art : Data, Charts, and Maps for ...","No matter what your actual job title, you are-...",/Truthful-Art-Alberto-Cairo/9780321934079,New Riders Publishing,2016-02-28,4.30,en,"Rolindale, MA, United States",Paperback,NaN,NaN
1,812995708,Dreyer's English : An Utterly Correct Guide to...,"NEW YORK TIMES BESTSELLER - A sharp, funny gra...",/Dreyers-English-Benjamin-Dreyer/9780812995701,Ballantine Books Inc.,2019-02-05,4.36,en,"London, United Kingdom",Spiral,NaN,NaN
2,134545133,The Adobe Photoshop Lightroom Classic CC Book ...,"Since Lightroom first launched 11 years ago, S...",/Adobe-Photoshop-Lightroom-Classic-CC-Book-for...,New Riders Publishing,2018-07-01,4.45,en,"Rolindale, MA, United States",Paperback,NaN,NaN
3,1430224894,Build Your Own CNC Machine,Do you like to build things? Are you ever frus...,/Build-Your-Own-CNC-Machine-James-Floyd-Kelly/...,aPress,2009-11-30,3.87,en,"Rockport, ME, United States",Paperback,NaN,NaN
4,8134367,Spelling Ages 5-6: New Edition,Level: KS1,/Spelling-Ages-5-6-New-Edition-Collins-Easy-Le...,Collins,2015-09-01,3.83,en,"North Adams, United States",Paperback,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
181648,1590176219,Proper Doctoring,"""People come to us for help. They come for hea...",/Proper-Doctoring-David-Mendel/9781590176214,NYRB Classics,2013-09-24,3.88,en,"New York, United States",Paperback,NaN,NaN
181649,415195624,Using Groups to Help People,This new edition of Using Groups to Help Peopl...,/Using-Groups-Help-People-Dorothy-Stock-Whitak...,ROUTLEDGE,2001-01-31,4.50,en,"London, United Kingdom",Paperback,NaN,NaN
181650,367437481,Dangerous Decade : Taiwan's Security and Crisi...,Taiwan's position looks increasingly precariou...,/Dangerous-Decade-Brendan-Taylor/9780367437480,ROUTLEDGE,2019-09-18,4.66,en,"London, United Kingdom",Paperback,NaN,NaN
181651,1781257736,Shapeshifters : A Doctor's Notes on Medicine &...,Unreliable bodies and shifting symptoms are al...,/Shapeshifters-Gavin-Francis/9781781257739,Wellcome Collection,2018-05-03,3.93,en,"London, United Kingdom",Hardback,NaN,NaN


In [0]:
prev_book_df = pd.read_csv('/content/drive/Shared drives/CIS550_Project/Books-temp/books_final2.csv', lineterminator='\n')

In [0]:
prev_book_df

,Unnamed: 0,Unnamed: 0.1,isbn10,title,description,url,publisher,publication_date,rating-avg,lang,img_url,place_name,format_name,num_pages
0,0,0,393325792,The Great Escape,They were American and British air force offic...,/Great-Escape-Paul-Brickhill/9780393325799,NaN,2004-08-17,4.24,en,NaN,"London, United Kingdom",Paperback,NaN
1,1,1,184454737X,Underbelly : The Gangland War,John Moran and Carl Williams were the two bigg...,/Underbelly-Andrew-Rule/9781844547371,John Blake,2009-03-13,3.59,en,http://books.google.com/books/content?id=FJOGP...,"North Adams, United States",Paperback,378.0
2,2,2,199669171,Oxford Guide to Plain English,"Plain English is the art of writing clearly, c...",/Oxford-Guide-Plain-English-Martin-Cutts/97801...,NaN,2013-09-15,4.18,en,NaN,"Wisconsin, United States",Paperback,NaN
3,3,3,1444185497,Get Talking and Keep Talking Portuguese Total ...,"When travelling, do you want to journey off th...",/Get-Talking-Keep-Talking-Portuguese-Total-Aud...,TEACH YOURSELF,2014-12-03,NaN,en,NaN,"North Adams, United States",Hardback,NaN
4,4,4,321934075,"The Truthful Art : Data, Charts, and Maps for ...","No matter what your actual job title, you are-...",/Truthful-Art-Alberto-Cairo/9780321934079,New Riders Publishing,2016-02-28,4.30,en,NaN,"Rolindale, MA, United States",Paperback,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226584,226584,226584,1592335799,Gluten-Free Entertaining : More Than 100 Natur...,This book includes 100 gluten-free recipes per...,/Gluten-Free-Entertaining-Olivia-Dupin/9781592...,NaN,2013-11-01,4.18,en,NaN,"Long Beach, CA, United States",Paperback,NaN
226585,226585,226585,754826635,"Fat Free : More Than 320 Tempting No Fat, Low ...",This title offers more than 320 tempting no fa...,/Fat-Free-Anne-Sheasby/9780754826637,LORENZ BOOKS,2013-06-15,2.00,en,NaN,"North Adams, United States",Spiral,NaN
226586,226586,226586,161379925X,Eating Healthy God's Way,PROVERBS 3:5 Trust in the Lord with all thine ...,/Eating-Healthy-Gods-Way-Oliver-Smith/97816137...,Xulon Press,2012-04-01,3.00,en,http://books.google.com/books/content?id=-lmrK...,[Place of publication not identified],Paperback,78.0
226587,226587,226587,9652293571,From Lokshen to Lo Mein : The Jewish Love Affa...,Some classic Jewish foods are analogous to Chi...,/From-Lokshen-Lo-Mein-Donald-I-Siegel/97896522...,Gefen Publishing House Ltd,2005-09-30,3.20,en,http://books.google.com/books/content?id=UzE1g...,"Milwaukee, United States",Paperback,224.0


In [0]:
inter_df = pd.merge(books_df, prev_book_df[['isbn10', 'num_pages', 'img_url']], left_on='isbn10', right_on='isbn10', how='inner')

In [0]:
# inter_df = inter_df.reset_index()
len(books_df) - len(inter_df)

138991

In [0]:
inter_df.drop(['num_pages_x', 'img_url_x'],axis=1, inplace=True)

In [0]:
inter_df.rename(columns={'num_pages_y': 'num_pages', 'img_url_y': 'img_url'}, inplace = True)

In [0]:
inter_df.to_csv('/content/drive/Shared drives/CIS550_Project/Books-temp/updated/working/book1.csv')
# inter_df

In [0]:
# books_df.reset_index(inplace=True)
books_df

In [0]:
df1 = books_df.copy()
df2 = inter_df.copy()

df1.set_index('isbn10', inplace = True)
df2.set_index('isbn10', inplace= True)

new_book_df = df1.drop(df2.index)

In [0]:
# new_book_df.reset_index(inplace= True)
new_book_df

,isbn10,title,description,url,publisher,publication-date,rating-avg,lang,place_name,format_name,num_pages,img_url
0,1844766586,Organic Kitchen,This book contains expert advice and fabulous ...,/Organic-Kitchen-Ysanne-Spevack/9781844766581,SOUTHWATER,2009-07-01,3.70,en,"North Adams, United States",Paperback,NaN,NaN
1,415276209,Managing Projects in Health and Social Care,Managing Projects in Health and Social Care is...,/Managing-Projects-Health-Social-Care-Vivien-M...,ROUTLEDGE,2002-08-01,4.00,en,"North Adams, United States",Paperback,NaN,NaN
2,262513412,Inner Presence : Consciousness as a Biological...,An overview and critical analysis of the study...,/Inner-Presence-Antti-Revonsuo/9780262513418,MIT Press,2009-09-30,4.20,en,"Zurich, Switzerland",Paperback,NaN,NaN
3,1472927796,Reeds Cooking at Sea Handbook,"Getting breakfast, lunch and dinner from tins ...",/Reeds-Cooking-at-Sea-Handbook-Sonja-Brodie/97...,THOMAS REED PUBLICATIONS,2017-05-16,5.00,en,"North Adams, United States",Paperback,NaN,NaN
4,1920434445,Abundance : City food from the Cape,At last another big cookbook from Marlene van ...,/Abundance-Marlene-van-der-Westhuizen/97819204...,Pan MacMillan,2014-09-02,4.71,en,"Urbana-Champaign, IL, United States",Paperback,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
138986,1590176219,Proper Doctoring,"""People come to us for help. They come for hea...",/Proper-Doctoring-David-Mendel/9781590176214,NYRB Classics,2013-09-24,3.88,en,"New York, United States",Paperback,NaN,NaN
138987,415195624,Using Groups to Help People,This new edition of Using Groups to Help Peopl...,/Using-Groups-Help-People-Dorothy-Stock-Whitak...,ROUTLEDGE,2001-01-31,4.50,en,"London, United Kingdom",Paperback,NaN,NaN
138988,367437481,Dangerous Decade : Taiwan's Security and Crisi...,Taiwan's position looks increasingly precariou...,/Dangerous-Decade-Brendan-Taylor/9780367437480,ROUTLEDGE,2019-09-18,4.66,en,"London, United Kingdom",Paperback,NaN,NaN
138989,1781257736,Shapeshifters : A Doctor's Notes on Medicine &...,Unreliable bodies and shifting symptoms are al...,/Shapeshifters-Gavin-Francis/9781781257739,Wellcome Collection,2018-05-03,3.93,en,"London, United Kingdom",Hardback,NaN,NaN


In [0]:
import math
step = 0

item = []
auth = {}
cat = {}

for i, row in new_book_df.iterrows():

  try:
    resp = req.get("https://www.googleapis.com/books/v1/volumes?q=isbn:"+ row.isbn10)
    resp = resp.json()

    row.num_pages = resp['items'][0]['volumeInfo'].get('pageCount')
    row.img_url = resp['items'][0]['volumeInfo'].get('imageLinks').get('thumbnail')
    
    authors = resp['items'][0]['volumeInfo'].get('authors')
    auth[row.isbn10] = authors

    categories = resp['items'][0]['volumeInfo'].get('categories')
    cat[row.isbn10] = categories
  except Exception as e:      
    print(str(e))
    item.append(row)
    continue
  # step += 1
  # if step > 2:
  #   break
  print(i)
  item.append(row)

Streaming output truncated to the last 5000 lines.
'items'
'items'
'items'
'items'
'items'
'items'
'items'
'items'
'items'
134001
'items'
134003
134004
'items'
'items'
'items'
134008
134009
'items'
'items'
'items'
'items'
'items'
'items'
'items'
134017
'items'
134019
'items'
134021
'items'
134023
'items'
'items'
'items'
134027
'items'
'items'
134030
'items'
'items'
134033
'items'
'items'
'items'
134037
'items'
'items'
134040
'items'
'items'
'items'
'items'
'items'
'items'
134047
134048
134049
'items'
'items'
134052
'items'
'items'
134055
'items'
'items'
'items'
134059
'items'
134061
134062
'items'
'items'
'items'
'items'
'items'
'items'
134069
134070
134071
'items'
134073
134074
'items'
'items'
'NoneType' object has no attribute 'get'
'items'
'items'
134080
134081
'items'
'items'
'items'
'items'
'items'
134087
134088
134089
'items'
134091
'items'
'items'
134094
'items'
134096
'items'
'items'
134099
'items'
'items'
134102
'items'
'items'
134105
'items'
'items'
'items'
'items'
'items'
'i

In [0]:
df = pd.DataFrame(item)

df

,isbn10,title,description,url,publisher,publication-date,rating-avg,lang,place_name,format_name,num_pages,img_url
0,1844766586,Organic Kitchen,This book contains expert advice and fabulous ...,/Organic-Kitchen-Ysanne-Spevack/9781844766581,SOUTHWATER,2009-07-01,3.70,en,"North Adams, United States",Paperback,128.0,http://books.google.com/books/content?id=6cBqP...
1,415276209,Managing Projects in Health and Social Care,Managing Projects in Health and Social Care is...,/Managing-Projects-Health-Social-Care-Vivien-M...,ROUTLEDGE,2002-08-01,4.00,en,"North Adams, United States",Paperback,NaN,NaN
2,262513412,Inner Presence : Consciousness as a Biological...,An overview and critical analysis of the study...,/Inner-Presence-Antti-Revonsuo/9780262513418,MIT Press,2009-09-30,4.20,en,"Zurich, Switzerland",Paperback,NaN,NaN
3,1472927796,Reeds Cooking at Sea Handbook,"Getting breakfast, lunch and dinner from tins ...",/Reeds-Cooking-at-Sea-Handbook-Sonja-Brodie/97...,THOMAS REED PUBLICATIONS,2017-05-16,5.00,en,"North Adams, United States",Paperback,NaN,NaN
4,1920434445,Abundance : City food from the Cape,At last another big cookbook from Marlene van ...,/Abundance-Marlene-van-der-Westhuizen/97819204...,Pan MacMillan,2014-09-02,4.71,en,"Urbana-Champaign, IL, United States",Paperback,208.0,http://books.google.com/books/content?id=ASgXm...
...,...,...,...,...,...,...,...,...,...,...,...,...
138986,1590176219,Proper Doctoring,"""People come to us for help. They come for hea...",/Proper-Doctoring-David-Mendel/9781590176214,NYRB Classics,2013-09-24,3.88,en,"New York, United States",Paperback,NaN,NaN
138987,415195624,Using Groups to Help People,This new edition of Using Groups to Help Peopl...,/Using-Groups-Help-People-Dorothy-Stock-Whitak...,ROUTLEDGE,2001-01-31,4.50,en,"London, United Kingdom",Paperback,NaN,NaN
138988,367437481,Dangerous Decade : Taiwan's Security and Crisi...,Taiwan's position looks increasingly precariou...,/Dangerous-Decade-Brendan-Taylor/9780367437480,ROUTLEDGE,2019-09-18,4.66,en,"London, United Kingdom",Paperback,NaN,NaN
138989,1781257736,Shapeshifters : A Doctor's Notes on Medicine &...,Unreliable bodies and shifting symptoms are al...,/Shapeshifters-Gavin-Francis/9781781257739,Wellcome Collection,2018-05-03,3.93,en,"London, United Kingdom",Hardback,282.0,http://books.google.com/books/content?id=0UOWt...


In [0]:
df.to_csv('/content/drive/Shared drives/CIS550_Project/Books-temp/updated/working/book2.csv')

In [0]:
import pickle 

with open('/content/drive/Shared drives/CIS550_Project/Books-temp/updated/working/bauthors.pickle', 'wb') as f:
  pickle.dump(auth, f)

In [0]:
import pickle 

with open('/content/drive/Shared drives/CIS550_Project/Books-temp/updated/working/bcat.pickle', 'wb') as f:
  pickle.dump(cat, f)

In [0]:
import pickle 

with open('/content/drive/Shared drives/CIS550_Project/Books-temp/updated/working/bitem.pickle', 'wb') as f:
  pickle.dump(item, f)